In [5]:
import geopandas as gpd
import rasterio
from rasterio.crs import CRS
from rasterio.warp import calculate_default_transform, reproject, Resampling
from rasterio.crs import CRS
from rasterio.io import MemoryFile
from rasterstats import zonal_stats
from rasterio.mask import mask
import numpy as np
import pandas as pd
import networkx as nx
from shapely.ops import unary_union
from shapely.strtree import STRtree
from shapely.geometry import mapping
import tempfile
import os
import sys
import logging

# Add the project root to sys.path so we can import from Code.utils everywhere
project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

from Code.utils.utility import load_config, resolve_path
from Code.utils.spatial_utility import load_and_reproject

# Load configuration
config = load_config()

for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)


logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s %(levelname)s: %(message)s',
    handlers=[
        logging.FileHandler("trim_debug.log", mode='w'),  # overwrite each time
        logging.StreamHandler()  # optional: show in notebook output
    ]
)

logger = logging.getLogger(__name__)


In [17]:
from rasterio.mask import mask
from rasterio.crs import CRS
from rasterio.io import MemoryFile
import numpy as np
import time

def save_masked_raster(array, transform, meta, out_path):
    meta_out = meta.copy()
    meta_out.update({
        'driver': 'GTiff',
        'dtype': array.dtype,
        'height': array.shape[0],
        'width': array.shape[1],
        'transform': transform,
        'crs': meta['crs'],
        'count': 1
    })
    logger.info(f"💾 Writing to: {out_path}")
    with rasterio.open(out_path, 'w', **meta_out) as dst:
        dst.write(array, 1)

def trim_aei_to_study_area():
    logger.info(f"⏱️ Function entry time: {time.time()}")

    # Reproject study area to match raster CRS (EPSG:4326)
    ssa_arid_shp_fp = resolve_path(config['SSA_Arid_by_Country_shp_path'])
    study_area = load_and_reproject(ssa_arid_shp_fp, target_crs="EPSG:4326")
    logger.info(f"Study area CRS: {study_area.crs}")
    study_area_union = study_area.geometry.unary_union

    aei_years = [2000]

    for year in aei_years:
        logger.info(f"🔁 Processing year: {year}")
        raster_key = f"Africa_AEI_{year}_asc_path"
        irrig_raster_path = resolve_path(config[raster_key])
        output_key = f"Irrigation_Arid_SSA_{year}_tif_path"
        output_path = resolve_path(config[output_key])

        # Load AEI raster and assign CRS if missing
        with rasterio.open(irrig_raster_path) as src_file:
            if src_file.crs is None:
                logger.info(f"⚠️ AEI raster for {year} missing CRS. Forcing EPSG:4326.")
                patched_meta = src_file.meta.copy()
                patched_meta['crs'] = CRS.from_epsg(4326)
                mem = MemoryFile()
                with mem.open(**patched_meta) as patched_src:
                    patched_src.write(src_file.read(1), 1)
                src = mem.open()
            else:
                src = src_file

            meta = src.meta.copy()

            # Mask to study area (EPSG:4326)
            masked, out_transform = mask(
                src,
                [study_area_union],
                crop=True,
                all_touched=True,
                nodata=src.nodata
            )

        print("Raster bounds:", src.bounds)
        print("Study area bounds:", study_area_union.bounds)

        array = masked[0]
        assert array.ndim == 2, f"Expected 2D array, got shape {array.shape}"
        logger.info(f"📐 Masked array shape: {array.shape}")
        
        meta.update({
            "transform": out_transform,
            "height": array.shape[0],
            "width": array.shape[1]
        })

        print("Min:", np.min(array))
        print("Max:", np.max(array))

        save_masked_raster(array, out_transform, meta, output_path)
        logger.info(f"✅ Trimmed AEI raster for {year} (all_touched) saved to: {output_path}")

from rasterio.mask import mask
from rasterio.crs import CRS
from rasterio.io import MemoryFile
import numpy as np
import time

def save_masked_raster(array, transform, crs, out_path):
    meta_out = {
        'driver': 'GTiff',
        'dtype': array.dtype,
        'height': array.shape[0],
        'width': array.shape[1],
        'transform': transform,
        'crs': crs,
        'count': 1
    }
    logger.info(f"💾 Writing to: {out_path}")
    with rasterio.open(out_path, 'w', **meta_out) as dst:
        dst.write(array, 1)

def reproject_to_4326(src):
    dst_crs = CRS.from_epsg(4326)
    transform, width, height = calculate_default_transform(
        src.crs, dst_crs, src.width, src.height, *src.bounds
    )

    kwargs = src.meta.copy()
    kwargs.update({
        'crs': dst_crs,
        'transform': transform,
        'width': width,
        'height': height
    })

    memfile = MemoryFile()
    with memfile.open(**kwargs) as dst:
        reproject(
            source=rasterio.band(src, 1),
            destination=rasterio.band(dst, 1),
            src_transform=src.transform,
            src_crs=src.crs,
            dst_transform=transform,
            dst_crs=dst_crs,
            resampling=Resampling.nearest
        )
    return memfile.open()

def trim_aei_to_study_area():
    logger.info(f"⏱️ Function entry time: {time.time()}")
    study_area = load_and_reproject(
        resolve_path(config['SSA_Arid_by_Country_shp_path']),
        target_crs="EPSG:4326"
    )
    logger.info(f"Study area CRS: {study_area.crs}")
    study_area_union = study_area.geometry.unary_union
    study_bounds = study_area_union.bounds

    aei_years = [2000]  # You can expand this list later

    for year in aei_years:
        logger.info(f"🔁 Processing year: {year}")
        irrig_raster_path = resolve_path(config[f"Africa_AEI_{year}_asc_path"])
        output_path = resolve_path(config[f"Irrigation_Arid_SSA_{year}_tif_path"])

        with rasterio.open(irrig_raster_path) as src:
            crs = src.crs or CRS.from_epsg(4326)
            if src.crs is None:
                logger.info(f"⚠️ Missing CRS. Forcing EPSG:4326")
                patched_meta = src.meta.copy()
                patched_meta['crs'] = crs
                mem = MemoryFile()
                with mem.open(**patched_meta) as patched:
                    patched.write(src.read(1), 1)
                src = mem.open()

            # Reproject to EPSG:4326 if needed
            if CRS.from_user_input(src.crs).to_epsg() != 4326:
                logger.info(f"🌍 Reprojecting AEI raster {year} to EPSG:4326")
                src = reproject_to_4326(src)

            # Log and check bounds BEFORE masking
            raster_bounds = src.bounds
            logger.info(f"🧭 Raster bounds: {raster_bounds}")
            logger.info(f"🧭 Study area bounds: {study_bounds}")

            if not (
                raster_bounds.right >= study_bounds[0] and
                raster_bounds.left <= study_bounds[2] and
                raster_bounds.top >= study_bounds[1] and
                raster_bounds.bottom <= study_bounds[3]
            ):
                logger.warning(
                    f"⚠️ Study area appears outside raster bounds. Skipping year {year}."
                )
                continue

            # Mask with study area
            masked, out_transform = mask(
                src,
                [study_area_union],
                crop=True,
                all_touched=True,
                nodata=src.nodata
            )

            array = masked[0]
            logger.info(f"📐 Masked array shape: {array.shape}")
            logger.info(f"📉 Min: {np.min(array)}, Max: {np.max(array)}")

            save_masked_raster(array, out_transform, src.meta.copy(), output_path)
            logger.info(f"✅ Trimmed AEI raster for {year} saved to: {output_path}")


In [15]:
trim_aei_to_study_area()

2025-07-17 11:10:01,027 INFO: ⏱️ Function entry time: 1752775801.0272772
2025-07-17 11:10:02,039 INFO: Study area CRS: EPSG:4326
2025-07-17 11:10:08,660 INFO: 🔁 Processing year: 2000
2025-07-17 11:10:10,338 INFO: 📐 Masked array shape: (746, 828)
2025-07-17 11:10:10,340 INFO: 💾 Writing to: /home/waves/data/Africa_Irrigation/Data/Processed/Irrigation_Arid_SSA_2000.tif


Raster bounds: BoundingBox(left=-180.0, bottom=-90.0, right=180.0, top=90.0)
Study area bounds: (-17.541666673165007, -34.83333327073998, 51.4156951904301, 27.2980709075929)
Min: -9.0
Max: 7504.133


2025-07-17 11:10:11,527 INFO: ✅ Trimmed AEI raster for 2000 (all_touched) saved to: /home/waves/data/Africa_Irrigation/Data/Processed/Irrigation_Arid_SSA_2000.tif


In [4]:
# --- Merge Overlapping Command Areas and Save as Shapefile ---

# Load command area shapefile
ca = load_and_reproject(resolve_path(config['No_Crop_Vectorized_Command_Area_shp_path']), target_crs="EPSG:3857")

# Build an undirected graph where nodes are polygon indices, edges mean overlap
G = nx.Graph()
G.add_nodes_from(range(len(ca)))
for i, geom1 in enumerate(ca.geometry):
    for j in range(i+1, len(ca)):
        geom2 = ca.geometry.iloc[j]
        if geom1.intersects(geom2):
            G.add_edge(i, j)

# Find connected components (groups of overlapping polygons)
groups = list(nx.connected_components(G))

# Merge polygons in each group
merged_geoms = []
n_merged = []
merged_gdw_ids = []
for group in groups:
    group_indices = list(group)
    group_df = ca.iloc[group_indices]
    merged_geom = unary_union(group_df.geometry)
    merged_geoms.append(merged_geom)
    n_merged.append(len(group_df))
    merged_gdw_ids.append(list(group_df['GDW_ID']))

CA_No_Overlap = gpd.GeoDataFrame({
    'geometry': merged_geoms,
    'n_merged': n_merged,
    'merged_GDW_IDs': [",".join(map(str, ids)) for ids in merged_gdw_ids]
}, crs=ca.crs)

# Check for overlaps in CA_No_Overlap
geoms = list(CA_No_Overlap.geometry)
overlap_found = False
for i, geom in enumerate(geoms):
    matches = [j for j, other in enumerate(geoms) if i != j and geom.intersects(other)]
    if matches:
        overlap_found = True
        print(f"Overlap found for geometry {i} (overlaps with: {matches})")
        break
if not overlap_found:
    output_path = resolve_path(config['No_Crop_Vectorized_CA_UniLayer_shp_path'])
    CA_No_Overlap.to_file(output_path, driver='ESRI Shapefile')
    print(f"Saved non-overlapping command areas to: {output_path}")
else:
    print('Overlaps detected in CA_No_Overlap!')

/tmp/ipykernel_4004600/168211551.py:47: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  CA_No_Overlap.to_file(output_path, driver='ESRI Shapefile')
2025-07-17 10:18:57,772 WARNING: Normalized/laundered field name: 'merged_GDW_IDs' to 'merged_GDW'


Saved non-overlapping command areas to: /home/waves/data/Africa_Irrigation/Data/Processed/No_Crop_Vectorized_UniLayer_CA-shp
